In [14]:
import math
import numpy as np
from statistics import stdev, harmonic_mean


In [1]:
from stockfish import Stockfish

# Initialize Stockfish (update the path accordingly)
stockfish = Stockfish(r"Stockfish\stockfish-windows-x86-64-avx2\stockfish\stockfish-windows-x86-64-avx2.exe")

# Test if the engine responds (by asking for a best move)
stockfish.set_fen_position("rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1")

try:
    best_move = stockfish.get_best_move()
    print("Stockfish is working!")
    print("Best move:", best_move)
    print("Evaluation:", stockfish.get_evaluation())
except Exception as e:
    print("Stockfish integration failed:", e)


Stockfish is working!
Best move: e2e4
Evaluation: {'type': 'cp', 'value': 37}


In [24]:
import pandas as pd
import chess
from stockfish import Stockfish

# Initialize Stockfish
stockfish = Stockfish(r"Stockfish\stockfish-windows-x86-64-avx2\stockfish\stockfish-windows-x86-64-avx2.exe")

# Load dataset
df = pd.read_csv("cleaned_games2.csv")

# Take the first game's moves
moves_str = df.loc[0, "Moves"]
moves_list = moves_str.split()

board = chess.Board()

def get_centipawn_loss(fen_before, move_uci):
    stockfish.set_fen_position(fen_before)
    eval_before = stockfish.get_evaluation()
    if eval_before["type"] == "cp":
        eval_before_value = eval_before["value"]
    else:
        eval_before_value = 10000 if eval_before["value"] > 0 else -10000

    stockfish.make_moves_from_current_position([move_uci])
    eval_after = stockfish.get_evaluation()
    if eval_after["type"] == "cp":
        eval_after_value = eval_after["value"]
    else:
        eval_after_value = 10000 if eval_after["value"] > 0 else -10000

    return abs(eval_before_value - eval_after_value)

data = []
for san_move in moves_list:
    try:
        move = board.parse_san(san_move)
        fen_before = board.fen()
        move_uci = move.uci()
        cp_loss = get_centipawn_loss(fen_before, move_uci)
        data.append({"Move": san_move, "Centipawn_Loss": cp_loss})
        board.push(move)
    except Exception as e:
        print(f"Error parsing move '{san_move}': {e}")
        break

result_df = pd.DataFrame(data)
print(result_df)
cp_list=[data[i]["Centipawn_Loss"] for i in range(len(data)) if i % 2 == 1]
print(cp_list)


    Move  Centipawn_Loss
0     e4               6
1     e5               1
2     d3              48
3     b6              87
4    Nf3              28
..   ...             ...
65  Qxh1              27
66  Bxh1             167
67  Rxg4              22
68   Be4             146
69    h3              54

[70 rows x 2 columns]
[1, 87, 11, 14, 32, 37, 35, 44, 22, 3, 2, 17, 1, 16, 2, 10, 0, 68, 45, 24, 87, 42, 17, 36, 0, 9262, 16, 11, 457, 52, 57, 18, 27, 22, 54]


In [25]:
print(sum(cp_list)) #1890
print(len(cp_list))

10629
35


In [23]:
import pandas as pd
import chess
import numpy as np
import math
from stockfish import Stockfish
from statistics import stdev, harmonic_mean

# Initialize Stockfish
stockfish = Stockfish(r"Stockfish\stockfish-windows-x86-64-avx2\stockfish\stockfish-windows-x86-64-avx2.exe")

# --- Utility functions ---

def centipawn_to_win_percent(cp):
    """Convert centipawn to Win% using Lichess formula."""
    return 50 + 50 * (2 / (1 + math.exp(-0.00368208 * cp)) - 1)

def move_accuracy(win_before, win_after):
    """Compute per-move accuracy (Lichess formula)."""
    return 103.1668 * math.exp(-0.04354 * abs(win_before - win_after)) - 3.1669

def bounded(val, lo, hi):
    return max(lo, min(hi, val))

def weighted_mean(values, weights):
    if not values or not weights or sum(weights) == 0:
        return None
    return sum(v * w for v, w in zip(values, weights)) / sum(weights)

# --- Load and prepare game ---

df = pd.read_csv("cleaned_games2.csv")
moves_str = df.loc[0, "Moves"]
moves_list = moves_str.split()
board = chess.Board()

centipawns = []
win_percents = []
accuracies = []

# --- Evaluate moves ---
for san_move in moves_list:
    try:
        move = board.parse_san(san_move)
        fen_before = board.fen()
        stockfish.set_fen_position(fen_before)
        eval_before = stockfish.get_evaluation()
        cp_before = eval_before["value"] if eval_before["type"] == "cp" else (10000 if eval_before["value"] > 0 else -10000)
        win_before = centipawn_to_win_percent(cp_before)

        stockfish.make_moves_from_current_position([move.uci()])
        eval_after = stockfish.get_evaluation()
        cp_after = eval_after["value"] if eval_after["type"] == "cp" else (10000 if eval_after["value"] > 0 else -10000)
        win_after = centipawn_to_win_percent(cp_after)

        acc = move_accuracy(win_before, win_after)

        centipawns.append(cp_after)
        win_percents.append(win_after)
        accuracies.append(acc)

        board.push(move)
    except Exception as e:
        print(f"Error parsing move '{san_move}': {e}")
        break

# --- Average Centipawn Loss ---
centipawn_diffs = [abs(centipawns[i] - centipawns[i-1]) for i in range(1, len(centipawns))]
avg_cpl = np.mean(centipawn_diffs) if centipawn_diffs else 0

# --- Compute overall game accuracy (Lichess-inspired) ---

# Sliding window for volatility
window_size = int(bounded(len(win_percents) / 10, 2, 8))
windows = [win_percents[i:i+window_size] for i in range(len(win_percents) - window_size + 1)]
weights = [bounded(stdev(w), 0.5, 12) if len(w) > 1 else 1 for w in windows]

# Align weights with move pairs
paired_acc = []
for i in range(len(win_percents) - 1):
    win_before = win_percents[i]
    win_after = win_percents[i + 1]
    acc = move_accuracy(win_before, win_after)
    w = weights[min(i, len(weights) - 1)]
    paired_acc.append((acc, w))

# Weighted mean and harmonic mean
weighted_acc = weighted_mean([a for a, _ in paired_acc], [w for _, w in paired_acc])
harmonic_acc = harmonic_mean([a for a, _ in paired_acc]) if paired_acc else 0
overall_acc = (weighted_acc + harmonic_acc) / 2 if weighted_acc else harmonic_acc

# --- Output ---
print("Average Centipawn Loss:", round(avg_cpl, 2))
print("Overall Game Accuracy%:", round(overall_acc, 2))


Average Centipawn Loss: 321.1
Overall Game Accuracy%: 87.94


In [27]:
import math
import pandas as pd
import chess
from statistics import stdev, harmonic_mean
from stockfish import Stockfish

# -----------------------
# CONFIG
# -----------------------
STOCKFISH_PATH = r"Stockfish\stockfish-windows-x86-64-avx2\stockfish\stockfish-windows-x86-64-avx2.exe"  # update if needed
CLIP_CP = 1000  # cap for centipawn evaluations (±1000)
STOCKFISH_DEPTH = 15

# Initialize wrapper Stockfish (no python-chess engine.popen_uci usage)
stockfish = Stockfish(STOCKFISH_PATH)
stockfish.set_depth(STOCKFISH_DEPTH)


# -----------------------
# UTILITIES
# -----------------------
def safe_get_eval_cp():
    """
    Return Stockfish evaluation in centipawns (White POV), clipped to ±CLIP_CP.
    Returns a float (clipped).
    In case of errors, returns None.
    """
    try:
        eval_data = stockfish.get_evaluation()
    except Exception:
        return None

    if eval_data is None:
        return None

    t = eval_data.get("type")
    v = eval_data.get("value")
    if t == "cp":
        cp = float(v)
    elif t == "mate":
        # map mate to a large cp then clip
        cp = 10000.0 if v > 0 else -10000.0
    else:
        return None

    # clip to avoid outliers (per discussion)
    if cp > CLIP_CP:
        cp = float(CLIP_CP)
    if cp < -CLIP_CP:
        cp = float(-CLIP_CP)
    return cp


def centipawn_to_winpercent(cp: float) -> float:
    """Lichess logistic conversion (Win% from White's perspective)."""
    # safe math: handle extreme cp values
    try:
        return 50 + 50 * (2 / (1 + math.exp(-0.00368208 * cp)) - 1)
    except OverflowError:
        return 100.0 if cp > 0 else 0.0


def move_accuracy_from_winperc(wb: float, wa: float) -> float:
    """Lichess single-move accuracy formula using Win% values."""
    diff = abs(wb - wa)
    return 103.1668 * math.exp(-0.04354 * diff) - 3.1669


def clamp(x, lo, hi):
    return max(lo, min(hi, x))


# -----------------------
# CORE: compute ACPL and overall accuracy for a given color
# -----------------------
def compute_acpl_and_accuracy_for_color(moves_str: str, color: str = "white", start_color: str = "white"):
    """
    moves_str: space-separated SAN moves (first ply = start_color moves)
    color: 'white' or 'black' -- which player's ACPL and overall accuracy to compute
    start_color: 'white' or 'black' (usually 'white' for typical PGNs)
    Returns: (avg_centipawn_loss_for_color, overall_accuracy_percent_for_color)
    """
    board = chess.Board()
    moves = moves_str.strip().split()
    if not moves:
        return 0.0, 0.0

    # We'll build a list of centipawn evals after each ply, and include initial eval as "initial"
    cp_after_each_ply = []

    # Set stockfish to start position
    stockfish.set_position([])

    # Evaluate initial position
    cp_init = safe_get_eval_cp()
    if cp_init is None:
        cp_init = 0.0
    # For Lichess-like pipeline we will collect all cp values starting with initial, then after each ply:
    cp_after_each_ply.append(cp_init)  # this corresponds to "Cp.initial" in scala

    # Also collect per-move data for ACPL for the requested color:
    # For ACPL we want the drop in evaluation from the perspective of the player who moved:
    # For White: loss = max(0, eval_before - eval_after) where evals are White POV (in cp)
    # For Black: loss = max(0, eval_after - eval_before) (i.e., from Black's POV)
    acpl_losses = []  # list of non-negative losses in centipawns for the requested color

    # iterate moves, keep stockfish and board synced by setting FEN prior to each move
    for ply_index, san in enumerate(moves):
        # parse SAN to UCI with python-chess (handles disambiguation)
        try:
            move = board.parse_san(san)
        except Exception:
            # illegal SAN — skip remaining moves
            break
        uci = move.uci()

        # Ensure stockfish is set at the same position as board before this ply
        fen_before = board.fen()
        try:
            stockfish.set_fen_position(fen_before)
        except Exception:
            # fallback: reset to start and replay up to this ply (robust but slower)
            stockfish.set_position([])
            for past_san in moves[:ply_index]:
                try:
                    m = board.copy(stack=False)  # not necessary but safe
                except Exception:
                    pass
            stockfish.make_moves_from_current_position(
                [board.parse_san(mv).uci() for mv in moves[:ply_index]]
            )
            # reset fen_before -> not re-evaluating here

        # safe eval before move
        cp_before = safe_get_eval_cp()
        if cp_before is None:
            cp_before = 0.0

        # apply move to stockfish
        try:
            stockfish.make_moves_from_current_position([uci])
        except Exception:
            # if stockfish cannot make move, try setting position via full move list:
            try:
                stockfish.set_position([m.uci() for m in map(lambda s: board.parse_san(s), moves[:ply_index] + [san])])
            except Exception:
                pass  # continue anyway

        # eval after move
        cp_after = safe_get_eval_cp()
        if cp_after is None:
            cp_after = cp_before  # fallback

        # append cp_after to list (this will be the cps list excluding initial in scala, but we built cp_after_each_ply including initial already)
        cp_after_each_ply.append(cp_after)

        # Determine which color made this move:
        mover_is_white = (ply_index % 2 == 0) if start_color == "white" else (ply_index % 2 == 1)
        mover_color = "white" if mover_is_white else "black"

        # Compute loss for the requested color, from that player's perspective
        if mover_color == color.lower():
            if color.lower() == "white":
                # loss only when position gets worse for white
                loss = max(0.0, cp_before - cp_after)
            else:
                # for black, evaluate from black POV: flip sign
                # black's eval before = -cp_before, after = -cp_after
                loss = max(0.0, (-cp_before) - (-cp_after))  # == max(0, cp_after - cp_before)
            acpl_losses.append(loss)

        # push move on python-chess board to keep it in sync for next SAN parse
        board.push(move)

    # ----- Average Centipawn Loss (for requested color) -----
    avg_cpl = float(sum(acpl_losses) / len(acpl_losses)) if acpl_losses else 0.0

    # ----- Overall Accuracy using Lichess Scala logic translated -----
    # In scala they do: allWinPercents = (Cp.initial :: cps).map(WinPercent.fromCentiPawns)
    # Here cp_after_each_ply already has initial then after every ply.
    all_win_percents = [centipawn_to_winpercent(cp) for cp in cp_after_each_ply]

    # windowSize = (cps.size / 10).atLeast(2).atMost(8)
    # Note: cps.size in scala is number of plies (excluding initial). We used len(cp_after_each_ply)-1
    num_cps = max(0, len(cp_after_each_ply) - 1)
    window_size = int(num_cps / 10) if num_cps > 0 else 0
    window_size = clamp(window_size, 2, 8)

    # build windows as in scala:
    # windows = List.fill(windowSize.atMost(allWinPercentValues.size) - 2)(allWinPercentValues.take(windowSize)).toList ::: allWinPercentValues.sliding(windowSize).toList
    all_vals = all_win_percents
    windows = []
    if window_size <= 0:
        windows = [[v] for v in all_vals]
    else:
        n_take = min(window_size, len(all_vals))
        num_prepends = max(0, n_take - 2)
        first_win = all_vals[:window_size]
        for _ in range(num_prepends):
            windows.append(first_win.copy())
        # sliding windows
        for i in range(0, len(all_vals) - window_size + 1):
            windows.append(all_vals[i:i + window_size])

    # weights = windows.map { xs => Maths.standardDeviation(xs).orZero.atLeast(0.5).atMost(12) }
    weights = []
    for xs in windows:
        if len(xs) >= 2:
            try:
                s = stdev(xs)
            except Exception:
                s = 0.0
        else:
            s = 0.0
        w = clamp(s if s is not None else 0.0, 0.5, 12.0)
        weights.append(w)

    # Now compute weightedAccuracies per adjacent pair (sliding 2 on all_win_percents),
    # zipped with weights and index, and collect entries for the player color.
    # For each pair (prev, next) at index i, determine color of the mover:
    weighted_entries = []  # list of tuples (accuracy, weight, mover_color)
    num_pairs = len(all_win_percents) - 1
    for i in range(num_pairs):
        prev = all_win_percents[i]
        nxt = all_win_percents[i + 1]
        # choose weight index: zip in scala pairs sliding2.zip(weights).zipWithIndex - weights list length should align with pairs;
        # to be safe, use min(i, len(weights)-1)
        w = weights[min(i, len(weights) - 1)] if weights else 1.0

        # Determine mover color for this transition (ply index i corresponds to the move that produced nxt)
        mover_is_white = (i % 2 == 0) if start_color == "white" else (i % 2 == 1)
        mover_col = "white" if mover_is_white else "black"

        # For accuracy computation we need Win% from mover's perspective:
        if mover_col == "white":
            win_before_player = prev  # prev is white POV already
            win_after_player = nxt
        else:
            # black's POV = 100 - white's Win%
            win_before_player = 100.0 - prev
            win_after_player = 100.0 - nxt

        acc = move_accuracy_from_winperc(win_before_player, win_after_player)
        weighted_entries.append((acc, w, mover_col))

    # Filter entries for the requested color and compute colorAccuracy as in scala:
    # colorAccuracy(color) = for weighted <- Maths.weightedMean(weightedAccuracies of that color)
    #                       harmonic <- Maths.harmonicMean(accuracies of that color)
    #                       yield AccuracyPercent((weighted + harmonic)/2)
    color_entries = [(a, w) for (a, w, c) in weighted_entries if c == color.lower()]
    if not color_entries:
        overall_accuracy = 0.0
    else:
        acc_vals = [a for a, _ in color_entries]
        acc_weights = [w for _, w in color_entries]
        # weighted mean (safe)
        weighted_mean_val = sum(a * w for a, w in color_entries) / sum(w for _, w in color_entries)
        # harmonic mean (safe)
        try:
            harmonic_val = harmonic_mean(acc_vals) if len(acc_vals) > 0 else 0.0
        except Exception:
            harmonic_val = sum(acc_vals) / len(acc_vals) if acc_vals else 0.0
        overall_accuracy = (weighted_mean_val + harmonic_val) / 2.0

    return avg_cpl, overall_accuracy


# -----------------------
# EXAMPLE: first row of cleaned_games_updated.csv
# -----------------------
df = pd.read_csv("cleaned_games_updated.csv")
first_moves = df.loc[4, "Moves"]

avg_cpl_white, overall_acc_white = compute_acpl_and_accuracy_for_color(first_moves, color="white", start_color="white")
avg_cpl_black, overall_acc_black = compute_acpl_and_accuracy_for_color(first_moves, color="black", start_color="white")

print(f"Average Centipawn Loss (White): {avg_cpl_white:.2f}")
print(f"Overall Accuracy (White): {overall_acc_white:.2f}%")
print(f"Average Centipawn Loss (Black): {avg_cpl_black:.2f}")
print(f"Overall Accuracy (Black): {overall_acc_black:.2f}%")


Average Centipawn Loss (White): 34.79
Overall Accuracy (White): 82.35%
Average Centipawn Loss (Black): 24.72
Overall Accuracy (Black): 87.04%
